In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load data
kc2 = pd.read_csv(r'/content/kc2_csv.csv')
print('kc2 shape: ', kc2.shape)
kc2.head()

kc2 shape:  (522, 22)


,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,lOCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,problems
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,...,2,2,2,2,1.2,1.2,1.2,1.2,1.4,no
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,...,1,1,1,1,1.0,1.0,1.0,1.0,1.0,yes
2,415.0,59.0,50.0,51.0,1159.0,8411.31,0.01,103.53,81.24,870848.58,...,359,35,9,10,47.0,106.0,692.0,467.0,106.0,yes
3,230.0,33.0,10.0,16.0,575.0,3732.82,0.03,39.82,93.74,148644.06,...,174,15,34,5,23.0,67.0,343.0,232.0,65.0,yes
4,175.0,26.0,12.0,13.0,500.0,3123.96,0.03,29.48,105.96,92103.07,...,142,7,19,4,18.0,58.0,310.0,190.0,51.0,yes


In [3]:
kc2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522 entries, 0 to 521
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   loc               522 non-null    float64
 1   v(g)              522 non-null    float64
 2   ev(g)             522 non-null    float64
 3   iv(g)             522 non-null    float64
 4   n                 522 non-null    float64
 5   v                 522 non-null    float64
 6   l                 522 non-null    float64
 7   d                 522 non-null    float64
 8   i                 522 non-null    float64
 9   e                 522 non-null    float64
 10  b                 522 non-null    float64
 11  t                 522 non-null    float64
 12  lOCode            522 non-null    int64  
 13  lOComment         522 non-null    int64  
 14  lOBlank           522 non-null    int64  
 15  lOCodeAndComment  522 non-null    int64  
 16  uniq_Op           522 non-null    float64
 1

In [4]:
target = 'problems'
y = kc2[target]
x=kc2.drop(['problems'], axis=1)
x.shape

(522, 21)

In [5]:
from sklearn.feature_selection import VarianceThreshold

var_thres=VarianceThreshold(threshold=1) # threshold=1 means it will tell features with 1 variance. We can change threshold value as per need
var_thres.fit(x)



VarianceThreshold(threshold=1)

In [6]:
var_thres.get_support() # We will remove false features 

array([ True,  True,  True,  True,  True,  True, False,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [7]:
remove_columns = [column for column in x.columns
                    if column not in x.columns[var_thres.get_support()]]

print(len(remove_columns))

print("name of columns to be removed: -")

for column in remove_columns:
    print(column)

2
name of columns to be removed: -
l
b


In [8]:
x.shape

(522, 21)

In [9]:
x=x.drop(remove_columns,axis=1)
# droping columns 

In [10]:
x.shape

(522, 19)

In [11]:
y.shape

(522,)

In [12]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=True)

In [13]:
print(x.shape)
print(y.shape)
print('-------------------')
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(522, 19)
(522,)
-------------------
(417, 19)
(417,)
(105, 19)
(105,)


In [14]:
from sklearn.svm import SVC
from sklearn import metrics #for checking the model accuracy
svc = SVC()
svc.fit(x_train, y_train)
svc_pred=svc.predict(x_test)
from sklearn import metrics #for checking the model accuracy
acc_svm = metrics.accuracy_score(svc_pred,y_test)
print('The accuracy of the SVM is:',acc_svm)
print('Th confusion matrix of SVM:\n ',metrics.confusion_matrix(y_test, svc_pred))

The accuracy of the SVM is: 0.8
Th confusion matrix of SVM:
  [[79  1]
 [20  5]]


In [15]:
print(metrics.confusion_matrix(y_test, svc_pred))
print(metrics.classification_report(y_test, svc_pred))

[[79  1]
 [20  5]]
              precision    recall  f1-score   support

          no       0.80      0.99      0.88        80
         yes       0.83      0.20      0.32        25

    accuracy                           0.80       105
   macro avg       0.82      0.59      0.60       105
weighted avg       0.81      0.80      0.75       105



In [16]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=2) #put 2 neighbors because we have 2 labels.
knn.fit(x_train,y_train)
knn_pred = knn.predict(x_test)
acc_knn = metrics.accuracy_score(knn_pred,y_test)
print('The accuracy of the KNN is', acc_knn)
print('The confusion matrix of KNN:\n ',metrics.confusion_matrix(y_test, knn_pred))
print(metrics.confusion_matrix(y_test, knn_pred))
print(metrics.classification_report(y_test, knn_pred))

The accuracy of the KNN is 0.7619047619047619
The confusion matrix of KNN:
  [[76  4]
 [21  4]]
[[76  4]
 [21  4]]
              precision    recall  f1-score   support

          no       0.78      0.95      0.86        80
         yes       0.50      0.16      0.24        25

    accuracy                           0.76       105
   macro avg       0.64      0.55      0.55       105
weighted avg       0.72      0.76      0.71       105



In [17]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(x_train, y_train)
y_predict = model.predict(x_test)
model_score = model.score(x_test, y_test)
print(model_score)
print(metrics.confusion_matrix(y_test, y_predict))
print(metrics.classification_report(y_test, y_predict))

0.7142857142857143
[[67 13]
 [17  8]]
              precision    recall  f1-score   support

          no       0.80      0.84      0.82        80
         yes       0.38      0.32      0.35        25

    accuracy                           0.71       105
   macro avg       0.59      0.58      0.58       105
weighted avg       0.70      0.71      0.71       105



In [18]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(x_train, y_train)
model.score(x_train, y_train) 

0.8369304556354916

In [19]:
test_pred = model.predict(x_test)
print(metrics.classification_report(y_test, test_pred))
print(metrics.confusion_matrix(y_test, test_pred))

              precision    recall  f1-score   support

          no       0.86      0.97      0.91        80
         yes       0.86      0.48      0.62        25

    accuracy                           0.86       105
   macro avg       0.86      0.73      0.76       105
weighted avg       0.86      0.86      0.84       105

[[78  2]
 [13 12]]
